In [ ]:
# define Spark client

from pyspark.sql import Row
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", "/usr/local/hadoop/warehouse") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hivemetastore:9083") \
    .config("spark.sql.catalog.iceberg.cache-enabled", False) \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

In [ ]:
# clean the table if it already exists

spark.sql("""
DROP TABLE IF EXISTS iceberg.default.bank_transfers PURGE
""")

In [ ]:
# create the table we will use in further excercises

spark.sql("""
CREATE TABLE IF NOT EXISTS iceberg.default.bank_transfers (
    id bigint COMMENT 'transfer id',
    amount int COMMENT 'transferred amount, expressed in cents',
    transferred_from string COMMENT 'initiator of the transfer',
    transferred_to string COMMENT 'receiver of the transfer',
    timestamp timestamp COMMENT 'time of the transfer'
) PARTITIONED BY (days(timestamp))
""")

In [ ]:
# describe the created table

spark.sql("""
DESCRIBE TABLE EXTENDED iceberg.default.bank_transfers
""").show(50, False)

In [ ]:
# insert data into the table using SQL

spark.sql("""
INSERT INTO iceberg.default.bank_transfers VALUES
    (1, 12000, "ACME INC",   "ASTROCORP",  TIMESTAMP"2022-11-14T00:55:00"),
    (2, 24000, "John Doe",   "Jane Doe",   TIMESTAMP"2022-11-15T02:11:00"),
    (3,   500, "Deborah S.", "Michael C.", TIMESTAMP"2022-11-17T16:25:07")
""")

# BONUS: Try inserting more data from this or another cell

In [ ]:
# insert more data to the table, using the DataFrame API

from datetime import datetime
from pyspark.sql import Row
from pyspark.sql.functions import to_timestamp


df = spark.createDataFrame([
    Row(
        id=4,
        amount=200,
        transferred_from="CTX Inc.",
        transferred_to="XYZ GmbH",
        timestamp=datetime.fromisoformat("2022-12-01T07:32:18")
    ),
])

df.writeTo("iceberg.default.bank_transfers").append()

# BONUS: Try inserting more data from this or another cell

In [ ]:
# check the correctness of the inserted data

spark.sql("""
SELECT * FROM iceberg.default.bank_transfers
ORDER BY id
""").show()

In [ ]:
# delete some data and verify that the table does not contain it anymore

spark.sql("""
DELETE FROM iceberg.default.bank_transfers
WHERE id=1
""")

spark.sql("""
SELECT * FROM iceberg.default.bank_transfers
""").show()

In [ ]:
# query the table again, this time using Dataframe API

df = spark.table("iceberg.default.bank_transfers")
df.show()

In [ ]:
# query the history of table commits

spark.sql("""
SELECT * FROM iceberg.default.bank_transfers.history
""").show(truncate=False)

In [ ]:
# query existing snapshots

spark.sql("""
SELECT snapshot_id, manifest_list FROM iceberg.default.bank_transfers.snapshots
""").show(truncate=False)

# BONUS: use SHOW COLUMNS ... to show all columns in this sub-table and query them as well

In [ ]:
# query existing manifests

spark.sql("""
SELECT path FROM iceberg.default.bank_transfers.manifests
""").show(truncate=False)

# BONUS: use SHOW COLUMNS ... to show all columns in this sub-table and query them as well

In [ ]:
# query metadata log entries

spark.sql("""
SELECT file FROM iceberg.default.bank_transfers.metadata_log_entries
""").show(truncate=False)

# BONUS: use SHOW COLUMNS ... to show all columns in this sub-table and query them as well